<a href="https://colab.research.google.com/github/yarusx/cat-vs-dogo/blob/main/cat_vs_dog_0_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

68608000/68606236 [==============================] - 1s 0us/step
Found 2000 files belonging to 2 classes.


In [ ]:
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 1000 files belonging to 2 classes.


In [ ]:
class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# for image, _ in train_dataset.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [ ]:
base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
loss0, accuracy0 = model.evaluate(validation_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

26/26 [==============================] - 1s 45ms/step - loss: 0.6649 - accuracy: 0.5408
initial loss: 0.66
initial accuracy: 0.54


In [ ]:
initial_epochs = 1
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
val_acc = history.history['val_accuracy']

63/63 [==============================] - 5s 77ms/step - loss: 0.6902 - accuracy: 0.6105 - val_loss: 0.4842 - val_accuracy: 0.6856


In [ ]:
while np.mean(val_acc)*100 < 98.5:
  initial_epochs = 3
  history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
  
  val_acc = history.history['val_accuracy']
  
try:
  !mkdir -p saved_model
except:
  pass

model.save('saved_model/dvc/')

!zip -r dvc.zip saved_model/dvc/

from google.colab import files
files.download("dvc.zip")

Epoch 1/3
63/63 [==============================] - 5s 75ms/step - loss: 0.1765 - accuracy: 0.9230 - val_loss: 0.0850 - val_accuracy: 0.9752
Epoch 2/3
63/63 [==============================] - 5s 74ms/step - loss: 0.1732 - accuracy: 0.9235 - val_loss: 0.0831 - val_accuracy: 0.9765
Epoch 3/3
63/63 [==============================] - 5s 73ms/step - loss: 0.1832 - accuracy: 0.9210 - val_loss: 0.0858 - val_accuracy: 0.9715
Epoch 1/3
63/63 [==============================] - 5s 77ms/step - loss: 0.1874 - accuracy: 0.9150 - val_loss: 0.0790 - val_accuracy: 0.9790
Epoch 2/3
63/63 [==============================] - 5s 75ms/step - loss: 0.1710 - accuracy: 0.9310 - val_loss: 0.0793 - val_accuracy: 0.9777
Epoch 3/3
63/63 [==============================] - 5s 73ms/step - loss: 0.1723 - accuracy: 0.9260 - val_loss: 0.0767 - val_accuracy: 0.9765
Epoch 1/3
63/63 [==============================] - 5s 73ms/step - loss: 0.1757 - accuracy: 0.9245 - val_loss: 0.0798 - val_accuracy: 0.9765
Epoch 2/3
63/63 [===

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q /content/drive/MyDrive/dvc.zip
dvc = tf.keras.models.load_model('/content/saved_model/dvc')

Mounted at /content/drive
replace saved_model/dvc/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
try:
  !mkdir -p saved_model
except:
  pass

model.save('saved_model/dvc/')

!zip -r dvc.zip saved_model/dvc/

from google.colab import files
files.download("dvc.zip")

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(160, 160))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 160, 160, 3)
	return img

img = load_image('/content/drive/MyDrive/dayana_1.JPG')

In [ ]:
categories = ["Cat", "Dog"]

In [ ]:
prediction = dvc_model.predict(img)
prediction = tf.nn.sigmoid(prediction)
print(prediction)
plt.figure()
plt.imshow(img[0]/255)
plt.title(categories[int(np.round_(prediction))])

In [ ]:
loss0, accuracy0 = model.evaluate(validation_dataset)

26/26 [==============================] - 1s 40ms/step - loss: 0.0848 - accuracy: 0.9765


In [ ]:
# #Retrieve a batch of images from the test set
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch).flatten()

# # Apply a sigmoid since our model returns logits
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(class_names[predictions[i]])
#   plt.axis("off")